In [3]:
import time

import numpy as np
from easyAI import TwoPlayerGame, AI_Player, Negamax, SSS, DUAL

In [4]:
class AIPlayerWithTimer(AI_Player):
    def __init__(self, AI_algo, name="AI"):
        super().__init__(AI_algo, name)
        self.move = {}
        self.mean_time = 0
        self.moves_counter = 0

    def ask_move(self, game):
        st = time.time()
        move = self.AI_algo(game)
        et = time.time()
        self.mean_time = ((self.mean_time * self.moves_counter) + (et - st)) / (self.moves_counter + 1)
        self.moves_counter += 1
        return move

In [5]:
class ClumsyConnectFour(TwoPlayerGame):
    def __init__(self, players, board=None):
        self.players = players
        self.board = (
            board if (board is not None) else (np.array([[0 for i in range(7)] for j in range(6)]))
        )
        self.current_player = 1
        self.pos_dir = np.array(
            [[[i, 0], [0, 1]] for i in range(6)]
            + [[[0, i], [1, 0]] for i in range(7)]
            + [[[i, 0], [1, 1]] for i in range(1, 3)]
            + [[[0, i], [1, 1]] for i in range(4)]
            + [[[i, 6], [1, -1]] for i in range(1, 3)]
            + [[[0, i], [1, -1]] for i in range(3, 7)]
        )

    def possible_moves(self):
        return [i for i in range(7) if (self.board[:, i].min() == 0)]

    def make_move(self, column):
        rand_move = np.random.choice(np.arange(-1, 2), p=[0.05, 0.9, 0.05])
        column = max(min(column + rand_move, 6), 0)
        line = np.argmin(self.board[:, column] != 0)
        self.board[line, column] = self.current_player

    def show(self):
        print(
            "\n"
            + "\n".join(
                ["0 1 2 3 4 5 6", 13 * "-"]
                + [
                    " ".join([[".", "O", "X"][self.board[5 - j][i]] for i in range(7)])
                    for j in range(6)
                ]
            )
        )

    def lose(self):
        return self.find_four(self.board, self.opponent_index)

    def is_over(self):
        return (self.board.min() > 0) or self.lose()

    def scoring(self):
        return -100 if self.lose() else 0

    def find_four(self, board, current_player):
        for pos, direction in self.pos_dir:
            streak = 0
            while (0 <= pos[0] <= 5) and (0 <= pos[1] <= 6):
                if board[pos[0], pos[1]] == current_player:
                    streak += 1
                    if streak == 4:
                        return True
                else:
                    streak = 0
                pos = pos + direction
        return False

In [6]:
class ConnectFour(TwoPlayerGame):
    def __init__(self, players, board=None):
        self.players = players
        self.board = (
            board
            if (board is not None)
            else (np.array([[0 for i in range(7)] for j in range(6)]))
        )
        self.current_player = 1
        self.pos_dir = np.array(
            [[[i, 0], [0, 1]] for i in range(6)]
            + [[[0, i], [1, 0]] for i in range(7)]
            + [[[i, 0], [1, 1]] for i in range(1, 3)]
            + [[[0, i], [1, 1]] for i in range(4)]
            + [[[i, 6], [1, -1]] for i in range(1, 3)]
            + [[[0, i], [1, -1]] for i in range(3, 7)]
        )

    def possible_moves(self):
        return [i for i in range(7) if (self.board[:, i].min() == 0)]

    def make_move(self, column):
        line = np.argmin(self.board[:, column] != 0)
        self.board[line, column] = self.current_player

    def show(self):
        print(
            "\n"
            + "\n".join(
                ["0 1 2 3 4 5 6", 13 * "-"]
                + [
                    " ".join([[".", "O", "X"][self.board[5 - j][i]] for i in range(7)])
                    for j in range(6)
                ]
            )
        )

    def lose(self):
        return self.find_four(self.board, self.opponent_index)

    def is_over(self):
        return (self.board.min() > 0) or self.lose()

    def scoring(self):
        return -100 if self.lose() else 0

    def find_four(self, board, current_player):
        for pos, direction in self.pos_dir:
            streak = 0
            while (0 <= pos[0] <= 5) and (0 <= pos[1] <= 6):
                if board[pos[0], pos[1]] == current_player:
                    streak += 1
                    if streak == 4:
                        return True
                else:
                    streak = 0
                pos = pos + direction
        return False

In [7]:
class Tournament:
    def __init__(self, tournament_length, algo1, algo2, prob):
        self.ranking = [0, 0, 0]
        self.tournament_length = tournament_length
        self.player1 = AIPlayerWithTimer(algo1)
        self.player2 = AIPlayerWithTimer(algo2)
        self.prob = prob

    def start_round(self, switch_player):
        if self.prob:
            game = ClumsyConnectFour([self.player1, self.player2])
        else:
            game = ConnectFour([self.player1, self.player2])
        if switch_player:
            game.switch_player()
        game.play(verbose=False)
        if game.lose():
            self.ranking[game.opponent_index - 1] += 1
        else:
            self.ranking[-1] += 1

    def start_tournament(self):
        for _ in range(self.tournament_length):
            switch_player = np.random.choice(np.arange(0, 2), p=[0.5, 0.5])
            self.start_round(switch_player)

# Wersja probabilistyczna

### Negamax (4) vs Negamax (2)

In [8]:
tournament = Tournament(40, Negamax(4), Negamax(2), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[27, 10, 3] 0.041497494463335005 0.005824729747570023


### Negamax (3) vs Negamax (5)

In [9]:
tournament = Tournament(40, Negamax(3), Negamax(5), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[20, 14, 6] 0.021857655455008874 0.1413909127495506


### DUAL (3) vs DUAL (4)

In [10]:
tournament = Tournament(40, DUAL(3), DUAL(4), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[15, 21, 4] 0.04530553335554146 0.06417854831499223


### DUAL (2) vs DUAL (5)

In [11]:
tournament = Tournament(40, DUAL(2), DUAL(5), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[18, 18, 4] 0.010571640542990663 0.17024938182698363


### Negamax (5) vs DUAL (5)

In [12]:
tournament = Tournament(40, Negamax(5), DUAL(5), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[24, 14, 2] 0.12116825559864873 0.1429909357433753


### Negamax (3) vs DUAL (3)

In [13]:
tournament = Tournament(40, Negamax(3), DUAL(3), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[23, 15, 2] 0.018559262465639232 0.024775442956876347


### SSS (3) vs SSS (5)

In [14]:
tournament = Tournament(40, SSS(3), SSS(5), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[16, 18, 6] 0.021976932440653872 0.1656387389490479


### SSS (2) vs SSS (4)

In [15]:
tournament = Tournament(40, SSS(2), SSS(4), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[19, 18, 3] 0.010803633456607516 0.059802484598091156


### SSS (3) vs Negamax (3)

In [16]:
tournament = Tournament(40, SSS(3), Negamax(3), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[17, 18, 5] 0.023212125486146762 0.017847455277734833


### SSS (4) vs DUAL (4)

In [17]:
tournament = Tournament(40, SSS(4), DUAL(4), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[21, 17, 2] 0.05061487204824423 0.8817893275379275


# Wersja deterministyczna

### Negamax (4) vs Negamax (2)

In [18]:
tournament = Tournament(40, Negamax(4), Negamax(2), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[25, 15, 0] 0.028505662867897436 0.004124908384523896


### Negamax (3) vs Negamax (5)

In [19]:
tournament = Tournament(40, Negamax(3), Negamax(5), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[20, 20, 0] 0.016310357420068047 0.10344166410596752


### DUAL (3) vs DUAL (4)

In [20]:
tournament = Tournament(40, DUAL(3), DUAL(4), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[23, 17, 0] 0.016686397135823882 0.03582534084811783


### DUAL (2) vs DUAL (5)

In [21]:
tournament = Tournament(40, DUAL(2), DUAL(5), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[18, 22, 0] 0.0056029448933357165 0.08651503912522138


### Negamax (5) vs DUAL (5)

In [22]:
tournament = Tournament(40, Negamax(5), DUAL(5), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[20, 20, 0] 0.08416786870440937 0.08880755031431044


### Negamax (3) vs DUAL (3)

In [23]:
tournament = Tournament(40, Negamax(3), DUAL(3), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[14, 26, 0] 0.013573017245844787 0.014818839336696425


### SSS (3) vs SSS (5)

In [24]:
tournament = Tournament(40, SSS(3), SSS(5), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[20, 20, 0] 0.01812837703807934 0.13028147381705218


### SSS (2) vs SSS (4)

In [25]:
tournament = Tournament(40, SSS(2), SSS(4), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[18, 22, 0] 0.007880021940987064 0.04597159183892923


### SSS (3) vs Negamax (3)

In [26]:
tournament = Tournament(40, SSS(3), Negamax(3), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[17, 23, 0] 0.019463243923689224 0.015411228882639039


### SSS (4) vs DUAL (4)

In [27]:
tournament = Tournament(40, SSS(4), DUAL(4), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[20, 20, 0] 0.03486426638232344 0.034472404917081194


### Negamax alfa-beta (4) vs Negamax alfa-beta (2)

In [28]:
tournament = Tournament(40, Negamax(4, win_score=3), Negamax(2, win_score=3), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[17, 23, 0] 0.032800183798137476 0.005050589850074371


### Negamax alfa-beta (3) vs SSS (4)

In [29]:
tournament = Tournament(40, Negamax(3, win_score=3), SSS(4), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[19, 21, 0] 0.01471055599681118 0.03639753176937542


### Negamax alfa-beta (3) vs DUAL (5)

In [30]:
tournament = Tournament(40, Negamax(3, win_score=3), SSS(5), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[14, 26, 0] 0.013494343284627395 0.11797355231267204


### Negamax alfa-beta (3) vs Negamax (3)

In [31]:
tournament = Tournament(40, Negamax(3, win_score=3), Negamax(3), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[18, 22, 0] 0.012153641801131399 0.012694538894452538


# NegaMax without alpha beta pruning

The negamax_no_alfa_beta algorithm is a variation of the negamax algorithm used for game playing. It uses the concept of a "transposition table" to avoid recomputing the same positions multiple times during the search.

The function takes in several parameters, including the current game state, the current depth in the game tree, and a function for evaluating the game state. It returns the best value found by the algorithm.

The function first checks if the current depth is 0 or if the game is over. If so, it returns the score for the current game state. Otherwise, it loops through all possible moves, calculating the move value for each using recursion.

## sounds great but wait, what alpha beta pruning actually is?
Alpha-beta pruning is a technique used in game playing algorithms to reduce the number of nodes that need to be evaluated. It works by eliminating parts of the game tree that are guaranteed to be worse than other parts.

### how does it work
The basic idea of alpha-beta pruning is to keep track of two values: alpha and beta. Alpha represents the best value found so far for the maximizing player, while beta represents the best value found so far for the minimizing player. As the search progresses, these values are updated based on the values of the nodes visited.
If at any point in the search, the current alpha value is greater than or equal to the current beta value, then we can stop evaluating the current branch of the tree, since it is guaranteed to be worse than another branch already evaluated. This is known as pruning.

### pros and cons
One of the main advantages of alpha-beta pruning is that it can significantly reduce the number of nodes that need to be evaluated during a game search. This can lead to faster and more efficient game playing algorithms.

However, alpha-beta pruning is not always guaranteed to provide optimal results. In some cases, it may prune a branch of the tree that leads to a better solution than the one found by the algorithm. Additionally, the effectiveness of alpha-beta pruning can depend on the ordering of the nodes in the game tree.

## implementation (NegaMax without alpha beta pruning)

In [32]:
LOWERBOUND, EXACT, UPPERBOUND = -1, 0, 1
inf = float('infinity')


def negamax_no_alfa_beta(game, depth, origDepth, scoring, tt=None):
    """
    Negamax algorithm with no alpha-beta pruning.

    Args:
        game: An object representing the game state.
        depth: The current depth in the game tree.
        origDepth: The original depth in the game tree.
        scoring: A function that evaluates the game state.
        tt: Transposition table for storing previous results.

    Returns:
        The best value found by the algorithm.
    """
    # Base case: if depth is 0 or the game is over, return the score

    if (depth == 0) or game.is_over():
        score = scoring(game)
        if score == 0:
            return score
        else:
            return score - 0.01 * depth * abs(score) / score
    # Get possible moves and current game state
    possible_moves = game.possible_moves()
    state = game
    best_move = possible_moves[0]
    if depth == origDepth:
        state.ai_move = possible_moves[0]

    bestValue = -inf
    unmake_move = hasattr(state, 'unmake_move')

    for move in possible_moves:

        if not unmake_move:
            game = state.copy()  # re-initialize move

        game.make_move(move)
        game.switch_player()

        move_value = -negamax_no_alfa_beta(game, depth - 1, origDepth, scoring, tt)

        if unmake_move:
            game.switch_player()
            game.unmake_move(move)

        if bestValue < move_value:
            bestValue = move_value
            best_move = move

    if tt is not None:
        assert best_move in possible_moves
        tt.store(game=state, depth=depth, value=bestValue,
                 move=best_move, flag=EXACT)

    return bestValue

In [33]:
class NegamaxNoAlfaBetaPruning:

    def __init__(self, depth, scoring=None, win_score=+inf, tt=None):
        self.scoring = scoring
        self.depth = depth
        self.tt = tt
        self.win_score = win_score

    def __call__(self, game):
        """
        Returns the AI's best move given the current state of the game.
        """
        scoring = self.scoring if self.scoring else (
            lambda g: g.scoring())  # horrible hack

        self.alpha = negamax_no_alfa_beta(game, self.depth, self.depth, scoring, self.tt)
        return game.ai_move

## comparison between NegaMax with and without alpha beta pruning

### deterministic model

#### NegaMax(3) vs NegamaxNoAlfaBetaPruning(3)

In [34]:
tournament = Tournament(40, Negamax(3, win_score=3), NegamaxNoAlfaBetaPruning(3), False)

tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[40, 0, 0] 0.018141466101216523 0.0832033688607423


#### NegaMax(5) vs NegamaxNoAlfaBetaPruning(5)

In [38]:
tournament = Tournament(40, Negamax(5, win_score=3), NegamaxNoAlfaBetaPruning(5), False)

tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[40, 0, 0] 0.13172518077649562 3.557136417759788


### probabilistic model

#### NegaMax(3) vs NegamaxNoAlfaBetaPruning(3)

In [ ]:
tournament = Tournament(40, Negamax(3, win_score=3), NegamaxNoAlfaBetaPruning(3), True)

tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

#### NegaMax(5) vs NegamaxNoAlfaBetaPruning(5)

In [ ]:
tournament = Tournament(40, Negamax(5, win_score=3), NegamaxNoAlfaBetaPruning(5), True)

tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

As we can see from the results above the version of NegaMax with alpha beta pruning is significantly faster than the version with the alpha beta pruning.

# expecti mini max with alpha beta pruning
## fine but a few words of description
### overall
ExpectiMiniMax with Alpha-Beta Pruning is a decision-making algorithm used in game theory and artificial intelligence to determine the best move in a game. It is an extension of the classic Minimax algorithm that incorporates chance nodes, i.e., nodes where the outcome of a move is determined by a random event.

In ExpectiMiniMax with Alpha-Beta Pruning, the algorithm iterates through the game tree, evaluating each possible move to a certain depth, similar to Minimax. However, at chance nodes, instead of simply returning the maximum or minimum value of its child nodes, the algorithm calculates a weighted average of the child node values, where the weights are the probabilities of each child node occurring.
### why alpha beta pruning you may ask
The algorithm uses alpha-beta pruning to improve its efficiency. Combining ExpectiMiniMax with Alpha-Beta Pruning allows the algorithm to evaluate each move efficiently while considering the possibility of chance events, ultimately leading to a better decision-making process.

Instead of the minimax values, the nodes have the expectimax values. They’re the same as the minimax values for MIN and MAX nodes, but for a chance node, the expectimax value is the expected value of its children
![Image](https://www.baeldung.com/wp-content/uploads/sites/4/2021/10/expected-value.jpg)


## and eventually the implementation!! :))

In [ ]:
class ExpectiMiniMaxAlphaBeta:

    def __init__(self, depth, scoring=None, win_score=float('inf'), tt=None):
        self.scoring = scoring
        self.depth = depth
        self.tt = tt
        self.win_score = win_score

    def __call__(self, game):
        """
        Returns the AI's best move given the current state of the game.
        """
        scoring = self.scoring if self.scoring else (
            lambda g: g.scoring())  # horrible hack

        self.alpha = -self.win_score
        self.beta = self.win_score
        self.best_move = None
        self.expecti_minimax(game, self.depth, scoring, self.alpha, self.beta)
        return self.best_move

    def expecti_minimax(self, game, depth, scoring, alpha, beta):
        """
        Expecti-miniMax algorithm with alpha-beta pruning.

        Args:
            game: An object representing the game state.
            depth: The current depth in the game tree.
            scoring: A function that evaluates the game state.
            alpha: The lower bound for the best value found so far.
            beta: The upper bound for the best value found so far.

        Returns:
            The best value found by the algorithm.
        """
        if depth == 0 or game.is_over():
            score = scoring(game)
            if score == 0:
                return score
            else:
                return score - 0.01 * depth * abs(score) / score

        possible_moves = game.possible_moves()
        state = game
        if depth == self.depth:
            self.best_move = possible_moves[0]

        best_value = -self.win_score

        for move in possible_moves:
            if not hasattr(state, 'unmake_move'):
                game = state.copy()
            else:
                game = state

            game.make_move(move)

            if game.is_over():
                value = scoring(game)
            else:
                value = -self.expecti_minimax(game, depth - 1, scoring, -beta, -alpha)

            if not hasattr(state, 'unmake_move'):
                game = state.copy()

            if value > best_value:
                best_value = value
                if depth == self.depth:
                    self.best_move = move

            alpha = max(alpha, value)
            if alpha >= beta:
                break

        return best_value


## probabilistic model

### ExpectiMiniMax(3) vs NegaMax(3)

In [ ]:
tournament = Tournament(40, ExpectiMiniMaxAlphaBeta(3, win_score=3), Negamax(3, 3), True)

tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

### ExpectiMiniMax(3) vs SSS(3)

In [ ]:
tournament = Tournament(40, ExpectiMiniMaxAlphaBeta(3, win_score=3), SSS(3), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

### ExpectiMiniMax(3) vs NegamaxNoAlfaBetaPruning(3)

In [ ]:
tournament = Tournament(40, ExpectiMiniMaxAlphaBeta(3, win_score=3), NegamaxNoAlfaBetaPruning(3), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

## deterministic model

### ExpectiMiniMax(3) vs NegaMax(3)

In [ ]:
tournament = Tournament(40, ExpectiMiniMaxAlphaBeta(3, win_score=3), Negamax(3, 3), False)

tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

### ExpectiMiniMax(3) vs SSS(3)

In [ ]:
tournament = Tournament(40, ExpectiMiniMaxAlphaBeta(3, win_score=3), SSS(3), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

### ExpectiMiniMax(3) vs NegamaxNoAlfaBetaPruning(3)

In [ ]:
tournament = Tournament(40, ExpectiMiniMaxAlphaBeta(3, win_score=3), NegamaxNoAlfaBetaPruning(3), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

# Wnioski
### Czasy
| Algorytm                 | Głębokość | Średni czas ruchu (s) \| Probabilistyczny | Średni czas ruchu (s) \| Deterministyczny |
|--------------------------|-----------|-------------------------------------------|-------------------------------------------|
| Negamax                  | 2         | 0.005                                     | 0.004                                     |
| Negamax                  | 3         | 0.015                                     | 0.01                                      |
| Negamax                  | 4         | 0.034                                     | 0.023                                     |
| Negamax                  | 5         | 0.093                                     | 0.067                                     |
| DUAL                     | 2         | 0.01                                      | 0.004                                     |
| DUAL                     | 3         | 0.023                                     | 0.012                                     |
| DUAL                     | 4         | 0.04                                      | 0.027                                     |
| DUAL                     | 5         | 0.1                                       | 0.07                                      |
| SSS                      | 2         | 0.008                                     | 0.005                                     |
| SSS                      | 3         | 0.018                                     | 0.014                                     |
| SSS                      | 4         | 0.037                                     | 0.029                                     |
| SSS                      | 5         | 0.167                                     | 0.1                                       |
| ExpectiMiniMax           | 3         | 0.027                                     | 0.031                                     |
| NegamaxNoAlfaBetaPruning | 3         | 0.061                                     | 0.068                                     |

### Wyniki
|                             | Negamax(2) | Negamax(3) | Negamax(4) | Negamax(5) | DUAL(3) | DUAL(4) | DUAL(5) |  SSS(2) |  SSS(3) |  SSS(4) |  SSS(5) | ExpectMiniMax(3) | NegamaxNoAlfaBetaPruning(3) |
|-----------------------------|:----------:|:----------:|:----------:|:----------:|:-------:|:-------:|:-------:|:-------:|:-------:|:-------:|:-------:|:----------------:|:---------------------------:|
| Negamax(2)                  |      X     |   20-16-4  |   16-20-4  |   16-19-5  | 21-14-5 | 15-18-7 | 16-20-4 | 16-20-4 | 18-18-4 | 24-12-4 | 20-19-1 |      36-3-1      |            30-6-4           |
| Negamax(3)                  |   19-16-5  |      X     |   20-16-4  |   15-23-2  | 23-13-4 | 23-14-3 | 22-13-5 | 17-14-9 | 20-18-2 | 19-19-2 | 18-18-4 |      38-2-0      |            31-6-3           |
| Negamax(4)                  |   18-18-4  |   20-14-6  |      X     |   19-18-3  | 16-23-1 | 14-17-9 | 18-16-6 | 17-18-5 | 22-14-4 | 23-15-2 | 22-15-3 |      29-9-2      |            33-4-3           |
| Negamax(5)                  |   21-16-3  |   18-17-5  |   21-14-5  |      X     | 21-11-8 | 22-16-2 | 23-13-4 | 20-18-2 | 21-12-7 | 16-19-5 | 19-17-4 |      29-8-3      |            34-5-1           |
| DUAL(3)                     |   16-21-3  |   13-26-1  |   18-20-2  |   9-25-6   |    X    | 19-21-0 | 19-16-5 | 17-21-2 | 15-21-4 | 12-26-2 | 15-21-4 |      28-7-5      |            29-7-4           |
| DUAL(4)                     |   18-17-5  |   14-22-4  |   21-15-4  |   14-25-1  | 19-16-5 |    X    | 16-21-3 | 23-12-5 | 21-14-5 | 21-15-4 | 12-22-1 |      28-9-3      |            33-6-1           |
| DUAL(5)                     |   19-16-5  |   17-18-5  |   10-23-7  |   18-21-1  | 28-10-2 | 16-20-4 |    X    | 16-17-7 | 19-13-8 | 16-22-2 | 25-13-2 |      35-5-0      |            31-8-1           |
| SSS(2)                      |   21-16-3  |  14-15-11  |   12-20-8  |   14-20-6  | 13-24-3 | 21-16-3 | 14-20-6 |    X    | 18-16-6 | 20-18-2 | 19-17-4 |      33-5-2      |            37-3-0           |
| SSS(3)                      |   18-17-5  |   15-21-4  |   13-23-4  |   16-17-7  | 15-20-5 | 16-21-3 | 19-18-3 | 22-15-3 |    X    | 14-23-3 | 22-11-7 |      34-4-2      |            33-5-2           |
| SSS(4)                      |   14-22-4  |   16-19-5  |   17-16-7  |   19-18-3  | 21-16-3 | 18-14-8 | 22-14-4 | 12-22-6 | 17-18-5 |    X    | 15-22-3 |      33-6-1      |            34-4-2           |
| SSS(5)                      |   11-22-7  |   12-24-4  |   23-15-2  |   17-19-4  | 17-18-5 | 19-17-4 | 16-16-8 | 22-15-3 | 17-15-8 | 20-16-4 |    X    |      34-6-0      |            30-9-1           |
| ExpectMiniMax(3)            |   7-30-3   |   9-30-1   |   3-31-6   |   5-33-2   | 12-28-0 |  5-34-1 |  8-30-2 |  6-33-1 |  9-30-1 |  5-31-4 |  7-31-2 |         X        |            30-9-1           |
| NegamaxNoAlfaBetaPruning(3) |   8-30-2   |   4-35-1   |   6-32-2   |   5-34-1   |  7-28-5 |  7-28-5 |  4-34-2 |  7-31-2 |  5-33-2 |  8-30-2 | 13-23-4 |      19-17-4     |              X              |

### Ranking algorytmów:
1. Negamax
2. DUAL
3. SSS
4. ExpectMiniMax
5. NegamaxNoAlfaBetaPruning